In [208]:
##import packages##
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import glob

In [209]:
#define path
path=r"/Users/valeriyamalenko/Desktop/trading/trading/momentum/data/mecklai-finance/"

#list paths to usd-term files
files=glob.glob(path +"/*-usd.csv")

#open the first file
tb=pd.read_csv(files[0])

#append all other files in the directory to the first file
for i in files[1:]:
    tb1=pd.read_csv(i, header=0)
    tb=tb.append(tb1)
    
tb=tb[['Currency', 'Date', 'SpotDate', 'SpotRate', 'col1']]
tb=tb.drop_duplicates()
tb=tb.reset_index()
tb.drop('index', inplace=True, axis='columns')

tb.SpotRate=1/tb.SpotRate
tb.col1=1/tb.col1

a=tb.Currency.str.split("/")
tb["Currency"]=a.str[1]+"/"+a.str[0]

In [210]:
#list paths to all usd-base files
files=glob.glob(path +"/usd-*.csv")

#open the first file
tb2=pd.read_csv(files[0])

#append all other files in the directory to the first file
for i in files[1:]:
    tb3=pd.read_csv(i, header=0)
    tb2=tb2.append(tb3)
    
tb2=tb2[['Currency', 'Date', 'SpotDate', 'SpotRate', 'col1']]
tb2=tb2.drop_duplicates()
tb2=tb2.reset_index()
tb2.drop('index', inplace=True, axis='columns')

In [211]:
#adding usd-term and usd-base dfs
frames=[tb,tb2]
df=pd.concat(frames)

In [212]:
##cleaning and transforming the data##

#convert date to datetime
df.Date=pd.to_datetime(df.Date)

#drop duplicates
ccy_l=df['Currency'].unique().tolist()
i=ccy_l[0]
df1=df[df['Currency']==i]
df1=df1.drop_duplicates(['Date'])

for i in ccy_l[1:]:
    df2=df[df['Currency']==i]
    df2=df2.drop_duplicates(['Date'])
    df1=df1.append(df2)
df=df1

#set date to be the index
df=df.set_index('Date')

#rename the column
df.rename(columns={'col1': 'fwd1m'}, inplace=True)

#delete missing data
df = df.loc[df["SpotRate"]!=0]
df = df.loc[df["fwd1m"]!=0]

#generate new cols for logs of spot and fwd rates
df['logSpotRate']=np.log(df.SpotRate)
df['logfwd1m']=np.log(df.fwd1m)

#select last day of the month for each month
df=df.loc[df.reset_index().groupby(df.index.to_period('M'))['Date'].max()]

#exclude data for 2019
dt = datetime.datetime(2019, 1, 1)
df=df[df.index < np.datetime64(dt)]

#checking the length (21*12*8 = 2016)
len(df)

2016

In [213]:
#transform pairs to be in per dollar terms
ccy_n=len(ccy_l)

#take a ccy data set
df1=df.loc[df['Currency']==ccy_l[0]]

#shift the fwd rate down
df1['shiftlogfwd1m']=df1.logfwd1m.shift(1)
#calculate return
df1['ret']=df1['shiftlogfwd1m']-df1['logSpotRate']

#repeat this for all ccies
for i in ccy_list[1:]:
    tb=df.loc[df['Currency']==i]
    tb['shiftlogfwd1m']=tb.logfwd1m.shift(1)
    tb['ret']=tb['shiftlogfwd1m']-tb['logSpotRate']
    #append all ccy dfs to form one df
    df1=df1.append(tb)    

/Users/valeriyamalenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/valeriyamalenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'ccy_list' is not defined